In [45]:
from typing import List
import string
import nltk
#nltk.download('averaged_perceptron_tagger')
#nltk.download('stopwords')
from nltk.corpus import stopwords


import sklearn
import pycrfsuite
from collections import defaultdict
from sklearn.metrics import classification_report

import spacy
from spacy.lang.en import English
from scipy import sparse
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from nerdata import Token, Chunk, LabeledSentence, chunks_from_bio_tag_seq, read_data, print_evaluation, print_output

print(sklearn.__version__)

0.24.2


In [13]:
# Load the training and development data.
train = read_data('../data/eng.train')
dev = read_data('../data/eng.testa')

In [18]:

def word2features(sent, i):
    
    features = defaultdict(int)
    word = sent.tokens[i].word
    stoplist = stopwords.words('english')
    
    features['bias'] = 1
    features['word.lower-' + word.lower()] = 1
    features['word.isupper'] = 1 if word.isupper() else 0
    features['word.ispunct'] = 1 if word[0] in string.punctuation else 0
    features['word.isnumber'] = 1 if word.isalpha() else 0
    features['word.length-' + str(len(word))] = 1
    features['word.isstopword'] = 1 if word[0] in stoplist else 0
    
    
    if i > 0:
        word1 = sent.tokens[i - 1].word
        features['-1:word.lower-' + word1.lower()] = 1
        features['-1:word.isupper'] = 1 if word1.isupper() else 0
        features['BOS'] = 0
    else:
        features['BOS'] = 1
        
    if i < len(sent) - 1:
        word1 = sent.tokens[i + 1].word
        postag1 = sent.tokens[i + 1].pos
        features['+1:word.lower-' + word1.lower()] = 1
        features['+1:word.isupper'] = 1 if word1.isupper() else 0
        features['EOS'] = 0
    else:
        features['EOS'] = 1
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return sent.bio_tags

def sent2words(sent):
    return [token.word for token in sent.tokens]

In [38]:
trainX = []
for s in train:
    for i in range(len(s)):
        trainX.append(s.tokens[i].word)
        
trainY = []
for s in train:
    for tag in s.bio_tags:
        trainY.append(tag)


devX = []
for s in dev:
    for i in range(len(s)):
        devX.append(s.tokens[i].word)
        
devY = []
for s in dev:
    for tag in s.bio_tags:
        devY.append(tag)

In [39]:
trainX_feat = []
for s in train:
    for i in range(len(s)):
        trainX_feat.append(word2features(s,i))

        
devX_feat = []
for s in dev:
    for i in range(len(s)):
        devX_feat.append(word2features(s,i))


In [40]:
# This helper function converts feature names to unique numerical IDs.

def create_vocab(examples):
    feature_vocab = {}
    idx = 0
    for example in examples:
        for feat in example:
            if feat not in feature_vocab:
                feature_vocab[feat] = idx
                idx += 1
                
    return feature_vocab

In [41]:
feature_vocab = create_vocab(trainX_feat)

In [42]:
len(feature_vocab)

61434

In [43]:
# This helper function converts a set of examples from a dictionary of feature names to values representation
# to a sparse representation of feature ids to values. This is important because almost all feature values will
# be 0 for most documents and it would be wasteful to save all in memory.

def features_to_ids(examples, feature_vocab):

    new_examples = sparse.lil_matrix((len(examples), len(feature_vocab)))
    for idx, example in enumerate(examples):
        for feat in example:
            if feat in feature_vocab:
                new_examples[idx, feature_vocab[feat]] = example[feat]
              
    return new_examples


In [46]:
trainX_ids = features_to_ids(trainX_feat, feature_vocab)
devX_ids = features_to_ids(devX_feat, feature_vocab)


In [47]:
lr_model = LogisticRegression(penalty = 'l2', C = 1.0, solver = 'lbfgs', max_iter = 1000)
lr_model.fit(trainX_ids, trainY)

print('Accuracy: %.3f' % lr_model.score(devX_ids, devY))

Accuracy: 0.950


accuracy after using the logistic regression algorithm is more than the crf model